In [0]:
%run "/Databricks Class/Tutorial"

### Basics

id,name,age
1,John,23
2,Mary,25
3,Bob,21


### Access Data 

### Utilitis

**dbutils.fs()**

[FileInfo(path='abfss://source@storagedatabricksclass.dfs.core.windows.net/Sales.csv', name='Sales.csv', size=869537, modificationTime=1764051323000)]

**dbutils.widgets**

### Write into delta format

In [0]:
df_csv.write.format('delta')\
        .mode('append')\
        .option('path', 'abfss://destination@storagedatabricksclass.dfs.core.windows.net/sales')\
        .save()

## Managed vs External Table

### Managed Table

#### Create Database

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS salesDb;

#### Create delta table

In [0]:
%sql
CREATE TABLE salesDb.mantable
(
  id INT,
  name STRING,
  age INT
)
USING DELTA

In [0]:
%sql
INSERT INTO salesDb.mantable VALUES
(1, "John", 20),
(2, "Mary", 30),
(3, "Mike", 40)

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
SELECT * from salesDb.mantable;

id,name,age
1,John,20
2,Mary,30
3,Mike,40


**When we drop table in managed delta table it will drop metastore layer and actual data that are located in managed data storage that are control in databricks**

In [0]:
%sql
DROP TABLE salesDb.mantable;

### External Table

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS salesDbext;

In [0]:
%sql
CREATE TABLE salesDbext.exttable
(
  id INT,
  name STRING,
  age INT
)
USING DELTA
LOCATION 'abfss://destination@storagedatabricksclass.dfs.core.windows.net/salesDb/exttable'

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8823020470256221>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "CREATE TABLE salesDbext.exttable\n(\n  id INT,\n  name STRING,\n  age INT\n)\nUSING DELTA\nLOCATION 'abfss://destination@storagedatabricksclass.dfs.core.windows.net/salesDb/exttable'\n")

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/db

In [0]:
%sql
INSERT INTO salesDbext.exttable VALUES
(1, 'John', 20),
(2, 'Mary', 30),
(3, 'Smith', 21)


In [0]:
%sql
SELECT * FROM salesDbext.exttable;

id,name,age
4,Ninja,20
5,Koto,30
6,Mere,21
1,John,20
2,Mary,30
3,Smith,21


#### If we drop the table in external table we only drop metadata but our actual data is in our control and that data is remain intact.

In [0]:
%sql
-- DROP TABLE salesDbext.exttable

## Delta table functionalities

**Insert**

In [0]:
%sql
INSERT INTO salesDbext.exttable VALUES
(4, 'Ninja', 20),
(5, 'Koto', 30),
(6, 'Mere', 21)

In [0]:
%sql
SELECT * FROM salesDbext.exttable;

id,name,age
4,Ninja,20
5,Koto,30
6,Mere,21
1,John,20
2,Mary,30
3,Smith,21


**Delete** 

In [0]:
%sql
DELETE FROM salesDbext.exttable
WHERE id = 6;

#### Data versioning

In [0]:
%sql
DESCRIBE HISTORY salesDbext.exttable;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
6,2025-11-26T05:49:32Z,144102568439169,bhaveshharmalkar28@gmail.com,RESTORE,"Map(version -> 2, timestamp -> null)",null,List(3186944715295030),1125-065034-dq07x317,5,Serializable,false,"Map(numRestoredFiles -> 0, removedFilesSize -> 0, numRemovedFiles -> 0, restoredFilesSize -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numOfFilesAfterRestore -> 2, tableSizeAfterRestore -> 2311)",null,Databricks-Runtime/17.3.x-scala2.13
5,2025-11-26T05:47:06Z,144102568439169,bhaveshharmalkar28@gmail.com,RESTORE,"Map(version -> 2, timestamp -> null)",null,List(3186944715295030),1125-065034-dq07x317,4,Serializable,false,"Map(numRestoredFiles -> 2, removedFilesSize -> 1217, numRemovedFiles -> 1, restoredFilesSize -> 2311, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numOfFilesAfterRestore -> 2, tableSizeAfterRestore -> 2311)",null,Databricks-Runtime/17.3.x-scala2.13
4,2025-11-26T05:27:36Z,144102568439169,bhaveshharmalkar28@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(3186944715295030),1125-065034-dq07x317,3,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 2311, p25FileSize -> 1217, numDeletionVectorsRemoved -> 1, minFileSize -> 1217, numAddedFiles -> 1, maxFileSize -> 1217, p75FileSize -> 1217, p50FileSize -> 1217, numAddedBytes -> 1217)",null,Databricks-Runtime/17.3.x-scala2.13
3,2025-11-26T05:27:29Z,144102568439169,bhaveshharmalkar28@gmail.com,DELETE,"Map(predicate -> [""(id#1456 = 6)""])",null,List(3186944715295030),1125-065034-dq07x317,2,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 5029, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 3150, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1860)",null,Databricks-Runtime/17.3.x-scala2.13
2,2025-11-26T05:12:02Z,144102568439169,bhaveshharmalkar28@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3186944715295030),1125-065034-dq07x317,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 1156)",null,Databricks-Runtime/17.3.x-scala2.13
1,2025-11-26T05:11:05Z,144102568439169,bhaveshharmalkar28@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3186944715295030),1125-065034-dq07x317,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 1155)",null,Databricks-Runtime/17.3.x-scala2.13
0,2025-11-26T05:11:03Z,144102568439169,bhaveshharmalkar28@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> false, properties -> {""delta.enableDeletionVectors"":""true"",""delta.writePartitionColumnsToParquet"":""true""}, statsOnLoad -> false)",null,List(3186944715295030),1125-065034-dq07x317,null,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-scala2.13


#### Time Travel

In [0]:
%sql
RESTORE TABLE salesDbext.exttable TO VERSION AS OF 2;

In [0]:
%sql
SELECT * FROM salesDbext.exttable;

id,name,age
4,Ninja,20
5,Koto,30
6,Mere,21
1,John,20
2,Mary,30
3,Smith,21


### Vacuum

**Below command take 7 days to delete the files from storage**

In [0]:
%sql
VACUUM salesDbext.exttable;

**Below command instantly delete the files**

In [0]:
%sql
VACUUM salesDbext.exttable RETAIN 0 HOURS;

### Delta Tables Optimization

**Optimize**

In [0]:
%sql
OPTIMIZE salesDbext.exttable;

path,metrics
abfss://destination@storagedatabricksclass.dfs.core.windows.net/salesDb/exttable,"List(1, 2, List(1228, 1228, 1228.0, 1, 1228), List(1155, 1156, 1155.5, 2, 2311), 0, null, null, 0, 1, 2, 0, true, 0, 0, 1764151309355, 1764151315227, 4, 1, null, List(0, 0), null, 3, 3, 225, 0, null)"


In [0]:
%sql
SELECT * FROM salesdbext.exttable

id,name,age
1,John,20
2,Mary,30
3,Smith,21
4,Ninja,20
5,Koto,30
6,Mere,21


**ZORDER BY()**

In [0]:
%sql
OPTIMIZE salesDbext.exttable ZORDER BY(id);

path,metrics
abfss://destination@storagedatabricksclass.dfs.core.windows.net/salesDb/exttable,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1228), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1764151567010, 1764151570188, 4, 0, null, List(0, 0), null, 3, 3, 0, 0, null)"


In [0]:
%sql
SELECT * FROM salesdbext.exttable;

id,name,age
1,John,20
2,Mary,30
3,Smith,21
4,Ninja,20
5,Koto,30
6,Mere,21


### Autoloader

In [0]:
df_auto = spark.readStream.format('cloudFiles')\
                .option('cloudFiles.format', 'parquet')\
                .option('cloudFiles.schemaLocation', 'abfss://aldestination@storagedatabricksclass.dfs.core.windows.net/checkpoint')\
                .load('abfss://alsource@storagedatabricksclass.dfs.core.windows.net')

In [0]:
df_auto.writeStream.format('delta')\
            .option('checkpointLocation', 'abfss://aldestination@storagedatabricksclass.dfs.core.windows.net/checkpoint')\
            .trigger(processingTime='5 seconds')\
            .start('abfss://aldestination@storagedatabricksclass.dfs.core.windows.net/data')